# Entender la incertidumbre en la clasificación multiclase con datos de resonancia magnética cerebral en transtornos de Alzhéimer y Demencia Frontotemporal 

In [2]:
import sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import GridSearchCV 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import brier_score_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
#import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

## 1.- Cargamos la base de datos

In [10]:
data = pd.read_csv("/Users/eguzkinehm/TFM/dataUnitat-2021-10-18_65years.csv")
data.drop(columns='Unnamed: 0.1', inplace=True)

## 2.- Filtramos los datos

Vamos a obtener únicamente los datos que no interesan para realizar la predicción del modelo. En este caso cogemos loa valores de los grosores corticales y los volúmenes subcorticales de los sujetos, además del grupo en el que han sido diagnósticados, su ID, el sexo y la edad en el momento de la realización de la resonancia magnética.

In [3]:
data = data[["ID", "Group", "Sex", "Age_at_MRI", "rh_middletemporal_thickness","rh_supramarginal_thickness","rh_superiortemporal_thickness",
          "rh_insula_thickness","rh_inferiortemporal_thickness","rh_inferiorparietal_thickness","rh_fusiform_thickness",
          "rh_bankssts_thickness","rh_precuneus_thickness","rh_lateralorbitofrontal_thickness",
          "rh_medialorbitofrontal_thickness","rh_superiorfrontal_thickness","rh_temporalpole_thickness",
          "rh_precentral_thickness","rh_entorhinal_thickness","rh_posteriorcingulate_thickness",
          "rh_parahippocampal_thickness","rh_parsopercularis_thickness","rh_isthmuscingulate_thickness",
          "rh_caudalmiddlefrontal_thickness","rh_postcentral_thickness","rh_superiorparietal_thickness",
          "rh_rostralmiddlefrontal_thickness","rh_parsorbitalis_thickness","rh_parstriangularis_thickness",
          "rh_lateraloccipital_thickness","rh_paracentral_thickness","rh_rostralanteriorcingulate_thickness",
          "rh_transversetemporal_thickness","rh_lingual_thickness","rh_cuneus_thickness","rh_frontalpole_thickness",
          "rh_pericalcarine_thickness","rh_caudalanteriorcingulate_thickness","lh_middletemporal_thickness","lh_supramarginal_thickness","lh_superiortemporal_thickness",
          "lh_insula_thickness","lh_inferiortemporal_thickness","lh_inferiorparietal_thickness","lh_fusiform_thickness",
          "lh_bankssts_thickness","lh_precuneus_thickness","lh_lateralorbitofrontal_thickness",
          "lh_medialorbitofrontal_thickness","lh_superiorfrontal_thickness","lh_temporalpole_thickness",
          "lh_precentral_thickness","lh_entorhinal_thickness","lh_posteriorcingulate_thickness",
          "lh_parahippocampal_thickness","lh_parsopercularis_thickness","lh_isthmuscingulate_thickness",
          "lh_caudalmiddlefrontal_thickness","lh_postcentral_thickness","lh_superiorparietal_thickness",
          "lh_rostralmiddlefrontal_thickness","lh_parsorbitalis_thickness","lh_parstriangularis_thickness",
          "lh_lateraloccipital_thickness","lh_paracentral_thickness","lh_rostralanteriorcingulate_thickness",
          "lh_transversetemporal_thickness","lh_lingual_thickness","lh_cuneus_thickness","lh_frontalpole_thickness",
          "lh_pericalcarine_thickness","lh_caudalanteriorcingulate_thickness", 'Left.Lateral.Ventricle',
 'Left.Cerebellum.Cortex',
 'Left.Thalamus.Proper',
 'Left.Caudate',
 'Left.Putamen',
 'Left.Pallidum',
 'Left.Hippocampus',
 'Left.Amygdala',
 'Right.Lateral.Ventricle',
 'Right.Cerebellum.Cortex',
 'Right.Thalamus.Proper',
 'Right.Caudate',
 'Right.Putamen',
 'Right.Pallidum',
 'Right.Hippocampus',
 'Right.Amygdala']]

Obtenemos información sobre los sujetos 

In [11]:
#Dividir las clases mujeres/hombres
print("0: Mujeres, 1: Hombres")
print(str(data.groupby('Sex').size()))
data_mujeres = data[data['Sex']==0]
data_hombres = data[data['Sex']==1]
print('Edad media en sujetos hombres: ' + str(np.mean(data_hombres.Age_at_MRI)) + '; Desviación de la edad en sujetos hombres: ' + str(np.std(data_hombres.Age_at_MRI)))
print('Edad media en sujetos mujeres: ' + str(np.mean(data_mujeres.Age_at_MRI)) + '; Desviación de la edad en sujetos mujeres: ' + str(np.std(data_mujeres.Age_at_MRI)))

print('Mujeres con AD: ' + str(len(data_mujeres[data_mujeres.Group=='AD'])) + '\n' + 'Mujeres con DFT: ' + str(len(data_mujeres[data_mujeres.Group=='DFT'])) + '\n' + 'Mujeres con CTR: ' + str(len(data_mujeres[data_mujeres.Group=='CTR'])))
print('Hombres con AD: ' + str(len(data_hombres[data_hombres.Group=='AD'])) + '\n' + 'Hombres con DFT: ' + str(len(data_hombres[data_hombres.Group=='DFT'])) + '\n' + 'Hombres con CTR: ' + str(len(data_hombres[data_hombres.Group=='CTR'])))

0: Mujeres, 1: Hombres
Sex
0     83
1    120
dtype: int64
Edad media en sujetos hombres: 56.62616666666668; Desviación de la edad en sujetos hombres: 7.061275402188727
Edad media en sujetos mujeres: 56.769759036144585; Desviación de la edad en sujetos mujeres: 6.268308696757047
Mujeres con AD: 35
Mujeres con DFT: 30
Mujeres con CTR: 18
Hombres con AD: 50
Hombres con DFT: 22
Hombres con CTR: 48


Edades de los difenretes grupos

In [10]:
grupos = data.groupby('Sex').size()
data_ctr = data[data['Group']=='CTR']
data_dft = data[data['Group']=='DFT']
data_ea = data[data['Group']=='AD']

print('Edad media en sujetos ctr: ' + str(np.mean(data_ctr.Age_at_MRI)) + '; Desviación de la edad en sujetos ctr: ' + str(np.std(data_ctr.Age_at_MRI)))
print('Edad media en sujetos dft: ' + str(np.mean(data_dft.Age_at_MRI)) + '; Desviación de la edad en sujetos dft: ' + str(np.std(data_dft.Age_at_MRI)))
print('Edad media en sujetos ea: ' + str(np.mean(data_ea.Age_at_MRI)) + '; Desviación de la edad en sujetos eaa: ' + str(np.std(data_ea.Age_at_MRI)))

Edad media en sujetos ctr: 54.95393939393938; Desviación de la edad en sujetos ctr: 8.34022125256767
Edad media en sujetos dft: 57.89076923076924; Desviación de la edad en sujetos dft: 4.798450810158919
Edad media en sujetos ea: 57.291176470588255; Desviación de la edad en sujetos eaa: 6.090288201770326


**Medidas medias por grupo**

In [69]:
pd.options.display.max_rows = None
grouped_data = data.groupby('Group')
grouped_data.mean().T

Group,AD,CTR,DFT
index,105.882353,193.954545,235.326923
Sex,0.588235,0.727273,0.423077
Age_at_MRI,57.291176,54.953939,57.890769
rh_middletemporal_thickness,2.649506,2.930273,2.683231
rh_supramarginal_thickness,2.357612,2.603394,2.455731
rh_superiortemporal_thickness,2.620306,2.830182,2.605481
rh_insula_thickness,2.856788,3.003636,2.733904
rh_inferiortemporal_thickness,2.679635,2.845636,2.652769
rh_inferiorparietal_thickness,2.224271,2.547424,2.444692
rh_fusiform_thickness,2.625129,2.785606,2.580731


## 3.- Elegimos contra qué grupos queremos probar

En este caso, como es una clasificación multiclase, lo que vamos a hacer es tener en cuenta los 3 grupos de pacientes: `CTR: Control`, `AD: Azhéimer Dementia` y `DFT: Demencia Frontotemporal`

In [7]:
"""
data_y = data[['Group']].replace('CTR', 0)
data_y = data_y[['Group']].replace('AD', 1)
data_y = data_y.replace('DFT', 2)
"""
data_y = data[['Group']]
data_y.groupby('Group').size()
# 0: CTR
# 1: AD
# 2: DFT

Group
AD     85
CTR    66
DFT    52
dtype: int64

In [8]:
data_x = data.drop(columns=['Sex', 'Group', 'ID', 'index'])
#data_x.drop(columns='level_0', inplace=True)
data_x

,Age_at_MRI,rh_middletemporal_thickness,rh_supramarginal_thickness,rh_superiortemporal_thickness,rh_insula_thickness,rh_inferiortemporal_thickness,rh_inferiorparietal_thickness,rh_fusiform_thickness,rh_bankssts_thickness,rh_precuneus_thickness,...,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
0,50.40,2.708,2.485,2.830,2.854,2.837,2.304,2.722,2.527,2.181,...,3568.7,1514.2,8078.5,49036.0,5811.5,3063.8,3922.1,1622.8,3713.5,1523.9
1,54.50,2.885,2.554,2.789,2.848,2.896,2.338,2.788,2.708,2.352,...,3766.4,1232.9,15697.9,66362.9,7469.7,3313.1,4815.9,2010.8,4049.4,1299.6
2,55.60,2.547,2.323,2.800,2.917,2.420,2.262,2.738,2.449,2.326,...,3201.7,953.3,6838.4,46305.3,5627.6,2172.5,3598.8,1397.6,3317.5,1143.0
3,56.00,2.601,2.276,2.586,3.002,2.778,2.012,2.754,2.058,1.895,...,3867.0,1415.4,13050.6,53563.9,6222.9,2780.2,4345.7,1931.3,4026.3,1568.0
4,56.20,2.381,2.251,2.236,2.768,2.549,1.974,2.339,1.886,1.984,...,2437.9,763.4,11940.8,36676.8,5464.2,2216.9,3415.6,1641.8,2461.0,871.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,64.66,3.213,2.771,3.116,2.915,3.181,2.685,3.077,2.746,2.497,...,3374.5,1188.7,5536.9,51583.3,6243.8,3275.7,4673.0,1729.7,3573.7,1464.1
199,61.68,2.735,2.469,2.576,3.065,2.996,2.513,2.648,2.365,2.263,...,3067.2,1160.1,10150.6,43022.6,5407.3,2354.3,2683.0,1453.0,3487.5,1380.4
200,32.77,2.992,2.579,2.944,3.068,2.889,2.588,3.022,2.898,2.530,...,4978.9,1700.6,6664.1,69157.7,7517.2,3974.0,4783.7,1964.1,4587.6,1743.1
201,60.58,2.732,2.490,2.724,3.106,2.801,2.470,2.724,2.624,2.510,...,4146.0,1619.2,4311.3,45697.9,6653.5,3020.6,4276.7,1915.3,4224.3,1402.7


## 4.- Entrenamos y evaluamos el modelo

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=42)

# Hacemos una copia de los datos
data_x_copia = data_x.copy()
data_y_copia = data_y.copy()

y_pred = pd.DataFrame(columns=['Group', 'AD_prob', 'CTR_prob', 'DFT_prob', 'Predict_Class'])
#Definimos el clasificador
clf = GaussianNB()

# Leave One Out
for i in data.index:
    X_test = data_x.iloc[[i]]
    y_test = data_y.loc[[i]]
    X_train = data_x.drop([i])
    y_train = data_y.drop([i])
    
    # Calibramos el modelo
    calibrated_clf  = CalibratedClassifierCV(clf,cv=10)
    
    #Entrenamos el modelo
    calibrated_clf.fit(X_train,y_train)
    
    y_pred_proba = calibrated_clf.predict_proba(X_test)
    y_pred_class = calibrated_clf.predict(X_test)
    
    # Vamos guardando los datos en un dataframe final
    
    = {'Group':y_test['Group'].values[0], 'AD_prob':y_pred_proba[0][0], 'CTR_prob':y_pred_proba[0][1], 'DFT_prob': y_pred_proba[0][2], 'Predict_Class':y_pred_class[0]}
    y_pred = y_pred.append(nueva_fila, ignore_index=True)
    
    data_y = data_y_copia
    data_x = data_x_copia


y_pred.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/y_pred.csv')

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. 

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

...

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/1488945713.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_pred = y_pred.append(nueva_fila, ignore_index=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Plea

In [10]:
y_pred

,Group,AD_prob,CTR_prob,DFT_prob,Predict_Class
0,AD,0.171298,0.668187,0.160514,CTR
1,AD,0.331023,0.500238,0.168739,CTR
2,AD,0.737131,0.092139,0.17073,AD
3,AD,0.736925,0.091891,0.171184,AD
4,AD,0.675071,0.095837,0.229092,AD
...,...,...,...,...,...
198,CTR,0.177566,0.662179,0.160255,CTR
199,AD,0.24552,0.099077,0.655403,DFT
200,CTR,0.170675,0.669349,0.159975,CTR
201,CTR,0.17542,0.664537,0.160043,CTR


**VALORACIÓN DE LAS PREDICCIONES DE CLASIFICACIÓN**

In [16]:
y_pred = pd.read_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/y_pred.csv')
y_pred['Group'] = y_pred['Group'].replace('AD', 'EA')
y_pred['Predict_Class'] = y_pred['Predict_Class'].replace('AD', 'EA')
df_y_pred_group = y_pred.groupby(['Group', 'Predict_Class']).size()
df_y_pred_group= pd.DataFrame(df_y_pred_group)
df_y_pred_group.rename(columns={0:'N'}, inplace=True)
#df_y_pred_group['percent'] = df_y_pred_group.groupby(['Group', 'Predict_Class']).size().sum().div(total_sum)*100

df_y_pred_group

N
Group Predict_Class    
CTR   CTR            61
      DFT             2
      EA              3
DFT   CTR            12
      DFT            28
      EA             12
EA    CTR            15
      DFT             3
      EA             67

### Obtenemos los mal clasificados

In [32]:
index_NOT_OK = y_pred.index[y_pred['Group'] != y_pred['Predict_Class']].tolist()
df_NOT_OK = y_pred[y_pred['Group']!=y_pred['Predict_Class']]
df_NOT_OK

,Unnamed: 0,Group,AD_prob,CTR_prob,DFT_prob,Predict_Class
0,0,AD,0.171298,0.668187,0.160514,CTR
1,1,AD,0.331023,0.500238,0.168739,CTR
16,16,AD,0.217045,0.097765,0.685190,DFT
35,35,AD,0.166991,0.673309,0.159700,CTR
40,40,AD,0.424098,0.100957,0.474945,DFT
45,45,AD,0.177355,0.660871,0.161774,CTR
46,46,AD,0.176654,0.662350,0.160995,CTR
47,47,AD,0.194252,0.642568,0.163179,CTR
49,49,AD,0.182049,0.656092,0.161859,CTR
50,50,AD,0.174765,0.664252,0.160983,CTR


Ahora con estos mal clasificados, vamos a obtener el dataframe del inicio para juntar todos los resultados y tener los valores thickness y volúmenes de las regiones cerebrales. 

En este paso también vamos a juntar todo el dataframe inicial con las probabilidades y las predicciones calculadas. De esta manera, vamos a poder obtener cuando queramos el dataframe con todos los valores predichos + volúmenes y thickness de las regiones cerebrales

In [24]:
all_index = y_pred.index.values

df_all_test = pd.DataFrame()
for i in all_index:
    df_all_test[i] = data.iloc[i]



df_all_test = df_all_test.T


df_all_test['Predict_Class'] = y_pred['Predict_Class']
df_all_test['AD_prob'] = y_pred['AD_prob']
df_all_test['CTR_prob'] = y_pred['CTR_prob']
df_all_test['DFT_prob'] = y_pred['DFT_prob']

df_all_test = df_all_test.reindex(columns=["ID", "Group", "Predict_Class", "AD_prob", "CTR_prob", "DFT_prob", "Sex", "Age_at_MRI", "rh_middletemporal_thickness","rh_supramarginal_thickness","rh_superiortemporal_thickness",
          "rh_insula_thickness","rh_inferiortemporal_thickness","rh_inferiorparietal_thickness","rh_fusiform_thickness",
          "rh_bankssts_thickness","rh_precuneus_thickness","rh_lateralorbitofrontal_thickness",
          "rh_medialorbitofrontal_thickness","rh_superiorfrontal_thickness","rh_temporalpole_thickness",
          "rh_precentral_thickness","rh_entorhinal_thickness","rh_posteriorcingulate_thickness",
          "rh_parahippocampal_thickness","rh_parsopercularis_thickness","rh_isthmuscingulate_thickness",
          "rh_caudalmiddlefrontal_thickness","rh_postcentral_thickness","rh_superiorparietal_thickness",
          "rh_rostralmiddlefrontal_thickness","rh_parsorbitalis_thickness","rh_parstriangularis_thickness",
          "rh_lateraloccipital_thickness","rh_paracentral_thickness","rh_rostralanteriorcingulate_thickness",
          "rh_transversetemporal_thickness","rh_lingual_thickness","rh_cuneus_thickness","rh_frontalpole_thickness",
          "rh_pericalcarine_thickness","rh_caudalanteriorcingulate_thickness","lh_middletemporal_thickness","lh_supramarginal_thickness","lh_superiortemporal_thickness",
          "lh_insula_thickness","lh_inferiortemporal_thickness","lh_inferiorparietal_thickness","lh_fusiform_thickness",
          "lh_bankssts_thickness","lh_precuneus_thickness","lh_lateralorbitofrontal_thickness",
          "lh_medialorbitofrontal_thickness","lh_superiorfrontal_thickness","lh_temporalpole_thickness",
          "lh_precentral_thickness","lh_entorhinal_thickness","lh_posteriorcingulate_thickness",
          "lh_parahippocampal_thickness","lh_parsopercularis_thickness","lh_isthmuscingulate_thickness",
          "lh_caudalmiddlefrontal_thickness","lh_postcentral_thickness","lh_superiorparietal_thickness",
          "lh_rostralmiddlefrontal_thickness","lh_parsorbitalis_thickness","lh_parstriangularis_thickness",
          "lh_lateraloccipital_thickness","lh_paracentral_thickness","lh_rostralanteriorcingulate_thickness",
          "lh_transversetemporal_thickness","lh_lingual_thickness","lh_cuneus_thickness","lh_frontalpole_thickness",
          "lh_pericalcarine_thickness","lh_caudalanteriorcingulate_thickness", 'Left.Lateral.Ventricle',
         'Left.Cerebellum.Cortex',
         'Left.Thalamus.Proper',
         'Left.Caudate',
         'Left.Putamen',
         'Left.Pallidum',
         'Left.Hippocampus',
         'Left.Amygdala',
         'Right.Lateral.Ventricle',
         'Right.Cerebellum.Cortex',
         'Right.Thalamus.Proper',
         'Right.Caudate',
         'Right.Putamen',
         'Right.Pallidum',
         'Right.Hippocampus',
         'Right.Amygdala'])

df_all_test

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all_test[i] = data.iloc[i]
/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all_test[i] = data.iloc[i]
/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor per

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all_test[i] = data.iloc[i]
/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all_test[i] = data.iloc[i]
/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor per

/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all_test[i] = data.iloc[i]
/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_all_test[i] = data.iloc[i]
/var/folders/cj/8md58kjn1j39_262mxnw8rxr0000gn/T/ipykernel_62688/3603246328.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor per

,ID,Group,Predict_Class,AD_prob,CTR_prob,DFT_prob,Sex,Age_at_MRI,rh_middletemporal_thickness,rh_supramarginal_thickness,...,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
0,sub-0001_01,AD,CTR,0.171298,0.668187,0.160514,1,50.4,2.708,2.485,...,3568.7,1514.2,8078.5,49036.0,5811.5,3063.8,3922.1,1622.8,3713.5,1523.9
1,sub-0002_01,AD,CTR,0.331023,0.500238,0.168739,0,54.5,2.885,2.554,...,3766.4,1232.9,15697.9,66362.9,7469.7,3313.1,4815.9,2010.8,4049.4,1299.6
2,sub-0003_01,AD,AD,0.737131,0.092139,0.17073,1,55.6,2.547,2.323,...,3201.7,953.3,6838.4,46305.3,5627.6,2172.5,3598.8,1397.6,3317.5,1143.0
3,sub-0004_01,AD,AD,0.736925,0.091891,0.171184,0,56.0,2.601,2.276,...,3867.0,1415.4,13050.6,53563.9,6222.9,2780.2,4345.7,1931.3,4026.3,1568.0
4,sub-0005_01,AD,AD,0.675071,0.095837,0.229092,1,56.2,2.381,2.251,...,2437.9,763.4,11940.8,36676.8,5464.2,2216.9,3415.6,1641.8,2461.0,871.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,sub-0355_01,CTR,CTR,0.177566,0.662179,0.160255,1,64.66,3.213,2.771,...,3374.5,1188.7,5536.9,51583.3,6243.8,3275.7,4673.0,1729.7,3573.7,1464.1
199,sub-0368_01,AD,DFT,0.24552,0.099077,0.655403,1,61.68,2.735,2.469,...,3067.2,1160.1,10150.6,43022.6,5407.3,2354.3,2683.0,1453.0,3487.5,1380.4
200,sub-0369_01,CTR,CTR,0.170675,0.669349,0.159975,0,32.77,2.992,2.579,...,4978.9,1700.6,6664.1,69157.7,7517.2,3974.0,4783.7,1964.1,4587.6,1743.1
201,sub-0370_01,CTR,CTR,0.17542,0.664537,0.160043,1,60.58,2.732,2.49,...,4146.0,1619.2,4311.3,45697.9,6653.5,3020.6,4276.7,1915.3,4224.3,1402.7


Guardamos esta tabla

In [39]:
df_all_test.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/all_Classification_date.csv')

In [10]:
df_all_test = pd.read_csv("/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/all_Classification_date.csv")

## Vamos a obtener un sujeto promedio por cada clase

### Agrupamos CTR - Obtenemos el sujeto medio de los controles

In [108]:
# De todo el dataframe -> filtramos los CTR
df_all_CTR = df_all_test[df_all_test['Group']=='CTR']
# Tenemos en cuenta únicamente los que se han clasificado bien para obtener el paciente "medio"
df_OK_CTR = df_all_CTR[df_all_CTR['Predict_Class']=='CTR']
df_OK_CTR.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/CTR/df_OK_CTR.csv')

#Guardamos también en un csv los controles mal clasificados para sacar gráficos más adelante
df_NOTOK_CTR = df_all_CTR[df_all_CTR['Predict_Class']!='CTR']
df_NOTOK_CTR.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/CTR/df_NOTOK_CTR.csv')

df_CTR_OK_mean_1 = df_OK_CTR.drop(columns=['Sex', 'Age_at_MRI'])
df_CTR_OK_mean = df_CTR_OK_mean_1.mean()
df_CTR_OK_mean = pd.DataFrame(df_CTR_OK_mean)
df_CTR_OK_mean = df_CTR_OK_mean.T
df_CTR_OK_mean.drop(columns = 'Unnamed: 0', inplace=True)
df_CTR_OK_mean.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/CTR/df_CTR_OK_mean.csv')
df_CTR_OK_mean

,AD_prob,CTR_prob,DFT_prob,rh_middletemporal_thickness,rh_supramarginal_thickness,rh_superiortemporal_thickness,rh_insula_thickness,rh_inferiortemporal_thickness,rh_inferiorparietal_thickness,rh_fusiform_thickness,...,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
0,0.182869,0.653374,0.163757,2.94559,2.614754,2.852377,3.021049,2.856738,2.554295,2.799492,...,3944.231148,1516.665574,8534.27541,51555.857377,6963.301639,3463.954098,4592.193443,1767.057377,4021.959016,1694.011475


In [107]:
df_all_CTR[df_all_CTR['Predict_Class']!='CTR']

,Unnamed: 0,ID,Group,Predict_Class,AD_prob,CTR_prob,DFT_prob,Sex,Age_at_MRI,rh_middletemporal_thickness,...,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
76,76,sub-0106_01,CTR,AD,0.718782,0.091362,0.189857,1,62.94,2.795,...,3727.5,1360.5,19110.7,55758.6,6273.3,3796.3,4690.6,2008.7,4280.2,1393.5
77,77,sub-0107_01,CTR,AD,0.715340,0.092620,0.192041,0,63.28,2.565,...,3555.5,1440.8,23991.7,45716.1,7167.0,3773.8,4270.6,2079.7,3546.1,1662.7
78,78,sub-0108_01,CTR,DFT,0.210808,0.106882,0.682311,0,63.95,2.947,...,3337.5,1603.6,44372.0,57285.7,6536.4,4815.9,5345.3,2028.7,3766.4,1720.7
125,125,sub-0164_01,CTR,DFT,0.341943,0.089769,0.568287,1,48.70,2.852,...,3139.4,955.1,15424.0,45572.7,5923.8,3312.6,3901.0,1934.1,3476.5,1341.3
141,141,sub-0190_01,CTR,AD,0.745373,0.085039,0.169588,1,59.48,2.558,...,3227.1,1220.4,5183.6,43788.5,5659.5,3021.0,4169.4,1395.0,3352.9,1574.0


#### Diferenciamos entre Thickness y Volúmenes

**1.- Thickness**

In [94]:
df_CTR_OK_mean_thickness = df_CTR_OK_mean.drop(columns=['Left.Lateral.Ventricle',
 'Left.Cerebellum.Cortex',
 'Left.Thalamus.Proper',
 'Left.Caudate',
 'Left.Putamen',
 'Left.Pallidum',
 'Left.Hippocampus',
 'Left.Amygdala',
 'Right.Lateral.Ventricle',
 'Right.Cerebellum.Cortex',
 'Right.Thalamus.Proper',
 'Right.Caudate',
 'Right.Putamen',
 'Right.Pallidum',
 'Right.Hippocampus',
 'Right.Amygdala'])
df_CTR_OK_mean_thickness

,AD_prob,CTR_prob,DFT_prob,rh_middletemporal_thickness,rh_supramarginal_thickness,rh_superiortemporal_thickness,rh_insula_thickness,rh_inferiortemporal_thickness,rh_inferiorparietal_thickness,rh_fusiform_thickness,...,lh_parstriangularis_thickness,lh_lateraloccipital_thickness,lh_paracentral_thickness,lh_rostralanteriorcingulate_thickness,lh_transversetemporal_thickness,lh_lingual_thickness,lh_cuneus_thickness,lh_frontalpole_thickness,lh_pericalcarine_thickness,lh_caudalanteriorcingulate_thickness
0,0.182869,0.653374,0.163757,2.94559,2.614754,2.852377,3.021049,2.856738,2.554295,2.799492,...,2.486836,2.308295,2.457443,2.777918,2.369984,2.027967,1.893705,2.804148,1.674852,2.682508


**2.- Volumes**

In [92]:
df_CTR_mean_volumes = df_CTR_mean.drop(columns=["rh_middletemporal_thickness","rh_supramarginal_thickness","rh_superiortemporal_thickness",
          "rh_insula_thickness","rh_inferiortemporal_thickness","rh_inferiorparietal_thickness","rh_fusiform_thickness",
          "rh_bankssts_thickness","rh_precuneus_thickness","rh_lateralorbitofrontal_thickness",
          "rh_medialorbitofrontal_thickness","rh_superiorfrontal_thickness","rh_temporalpole_thickness",
          "rh_precentral_thickness","rh_entorhinal_thickness","rh_posteriorcingulate_thickness",
          "rh_parahippocampal_thickness","rh_parsopercularis_thickness","rh_isthmuscingulate_thickness",
          "rh_caudalmiddlefrontal_thickness","rh_postcentral_thickness","rh_superiorparietal_thickness",
          "rh_rostralmiddlefrontal_thickness","rh_parsorbitalis_thickness","rh_parstriangularis_thickness",
          "rh_lateraloccipital_thickness","rh_paracentral_thickness","rh_rostralanteriorcingulate_thickness",
          "rh_transversetemporal_thickness","rh_lingual_thickness","rh_cuneus_thickness","rh_frontalpole_thickness",
          "rh_pericalcarine_thickness","rh_caudalanteriorcingulate_thickness","lh_middletemporal_thickness","lh_supramarginal_thickness","lh_superiortemporal_thickness",
          "lh_insula_thickness","lh_inferiortemporal_thickness","lh_inferiorparietal_thickness","lh_fusiform_thickness",
          "lh_bankssts_thickness","lh_precuneus_thickness","lh_lateralorbitofrontal_thickness",
          "lh_medialorbitofrontal_thickness","lh_superiorfrontal_thickness","lh_temporalpole_thickness",
          "lh_precentral_thickness","lh_entorhinal_thickness","lh_posteriorcingulate_thickness",
          "lh_parahippocampal_thickness","lh_parsopercularis_thickness","lh_isthmuscingulate_thickness",
          "lh_caudalmiddlefrontal_thickness","lh_postcentral_thickness","lh_superiorparietal_thickness",
          "lh_rostralmiddlefrontal_thickness","lh_parsorbitalis_thickness","lh_parstriangularis_thickness",
          "lh_lateraloccipital_thickness","lh_paracentral_thickness","lh_rostralanteriorcingulate_thickness",
          "lh_transversetemporal_thickness","lh_lingual_thickness","lh_cuneus_thickness","lh_frontalpole_thickness",
          "lh_pericalcarine_thickness","lh_caudalanteriorcingulate_thickness"])
df_CTR_mean_volumes

,AD_prob,CTR_prob,DFT_prob,Left.Lateral.Ventricle,Left.Cerebellum.Cortex,Left.Thalamus.Proper,Left.Caudate,Left.Putamen,Left.Pallidum,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
0,0.182869,0.653374,0.163757,9368.983607,50510.255738,7022.031148,3367.877049,4566.903279,1874.091803,3944.231148,1516.665574,8534.27541,51555.857377,6963.301639,3463.954098,4592.193443,1767.057377,4021.959016,1694.011475


**Agrupamos AD - Obtenemos el sujeto medio de los Alzhéimer**

In [98]:
df_all_AD = df_all_test[df_all_test['Group']=='AD']
df_OK_AD = df_all_AD[df_all_AD['Predict_Class']=='AD']
#guardamos los alzhéimers bien clasificados
df_OK_AD.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/AD/df_OK_AD.csv')

#guardamos los alzhéimers mal clasificados
df_NOTOK_AD = df_all_AD[df_all_AD['Predict_Class']!='AD']
df_NOTOK_AD.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/AD/df_NOTOK_AD.csv')

df_AD_OK_mean_1 = df_OK_AD.drop(columns=['Sex', 'Age_at_MRI'])
df_AD_OK_mean = df_AD_OK_mean_1.mean()
df_AD_OK_mean = pd.DataFrame(df_AD_OK_mean)
df_AD_OK_mean = df_AD_OK_mean.T
df_AD_OK_mean.drop(columns = 'Unnamed: 0', inplace=True)
df_AD_OK_mean.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/AD/df_AD_OK_mean.csv')
df_AD_OK_mean

,AD_prob,CTR_prob,DFT_prob,rh_middletemporal_thickness,rh_supramarginal_thickness,rh_superiortemporal_thickness,rh_insula_thickness,rh_inferiortemporal_thickness,rh_inferiorparietal_thickness,rh_fusiform_thickness,...,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
0,0.723539,0.100037,0.176424,2.605866,2.318701,2.586433,2.831045,2.640164,2.175015,2.580836,...,3320.78806,1171.176119,14202.873134,51710.659701,6509.252239,3079.485075,4128.570149,1857.91791,3413.502985,1340.870149


**Agrupamos DFT - Obtenemos el sujeto medio de los Demencia Frontotemporal**

In [102]:
df_all_DFT = df_all_test[df_all_test['Group']=='DFT']
df_OK_DFT = df_all_test[df_all_test['Predict_Class']=='DFT']
#guardamos los alzhéimers bien clasificados
df_OK_DFT.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/DFT/df_OK_DFT.csv')

#guardamos los alzhéimers mal clasificados
df_NOTOK_DFT = df_all_DFT[df_all_DFT['Predict_Class']!='DFT']
df_NOTOK_DFT.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/DFT/df_NOTOK_DFT.csv')

df_DFT_OK_mean_1 = df_OK_DFT.drop(columns=['Sex', 'Age_at_MRI'])
df_DFT_OK_mean = df_DFT_OK_mean_1.mean()
df_DFT_OK_mean = pd.DataFrame(df_DFT_OK_mean)
df_DFT_OK_mean = df_DFT_OK_mean.T
df_DFT_OK_mean.drop(columns = 'Unnamed: 0', inplace=True)
df_DFT_OK_mean.to_csv('/Users/eguzkinehm/TFM/TFM_code/Calibrator_Classifier/RESULTADOS/DFT/df_DFT_OK_mean.csv')
df_DFT_OK_mean

,AD_prob,CTR_prob,DFT_prob,rh_middletemporal_thickness,rh_supramarginal_thickness,rh_superiortemporal_thickness,rh_insula_thickness,rh_inferiortemporal_thickness,rh_inferiorparietal_thickness,rh_fusiform_thickness,...,Left.Hippocampus,Left.Amygdala,Right.Lateral.Ventricle,Right.Cerebellum.Cortex,Right.Thalamus.Proper,Right.Caudate,Right.Putamen,Right.Pallidum,Right.Hippocampus,Right.Amygdala
0,0.232237,0.094524,0.67324,2.559182,2.438485,2.515818,2.595667,2.542606,2.440242,2.449758,...,3016.439394,992.739394,19778.918182,49627.433333,6445.409091,3100.630303,3755.642424,1860.312121,3240.042424,1251.084848


In [87]:
df_DFT_mean_thickness = df_DFT_mean.drop(columns=['Left.Lateral.Ventricle',
 'Left.Cerebellum.Cortex',
 'Left.Thalamus.Proper',
 'Left.Caudate',
 'Left.Putamen',
 'Left.Pallidum',
 'Left.Hippocampus',
 'Left.Amygdala',
 'Right.Lateral.Ventricle',
 'Right.Cerebellum.Cortex',
 'Right.Thalamus.Proper',
 'Right.Caudate',
 'Right.Putamen',
 'Right.Pallidum',
 'Right.Hippocampus',
 'Right.Amygdala'])
df_DFT_mean_thickness

,AD_prob,CTR_prob,DFT_prob,rh_middletemporal_thickness,rh_supramarginal_thickness,rh_superiortemporal_thickness,rh_insula_thickness,rh_inferiortemporal_thickness,rh_inferiorparietal_thickness,rh_fusiform_thickness,...,lh_parstriangularis_thickness,lh_lateraloccipital_thickness,lh_paracentral_thickness,lh_rostralanteriorcingulate_thickness,lh_transversetemporal_thickness,lh_lingual_thickness,lh_cuneus_thickness,lh_frontalpole_thickness,lh_pericalcarine_thickness,lh_caudalanteriorcingulate_thickness
0,0.232237,0.094524,0.67324,2.559182,2.438485,2.515818,2.595667,2.542606,2.440242,2.449758,...,2.328606,2.305515,2.37803,2.538212,2.376758,2.005455,1.889455,2.687455,1.67303,2.671182


In [29]:
from sklearn.metrics import classification_report
print(classification_report(df_all_test['Group'], df_all_test['Predict_Class']))
# alta precisión pero bajo el recall: el modelo no detecta bien una de las clases pero cuando lo hace es altamente confiable
#baja precisión y alto recall: detecta bien esa clase pero también incluye muestras de la otra clase

              precision    recall  f1-score   support

          AD       0.82      0.79      0.80        85
         CTR       0.69      0.92      0.79        66
         DFT       0.85      0.54      0.66        52

    accuracy                           0.77       203
   macro avg       0.79      0.75      0.75       203
weighted avg       0.78      0.77      0.76       203

